# Segmentation

## Importing dependencies

In [ ]:
import numpy as np
from keras.models import load_model
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import openslide as op
from PIL import Image
from util import IoU, Dice, masked, remove_small_regions, normalize
from scipy import misc
import os
import time
import h5py
import configparser
import ast
from util import mkdirs

## Configuration

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')
section = 'segmentation'

In [ ]:
# Paths
filename = config.get(section, 'filename')
maskname = config.get(section, 'maskname')
matrice_file = h5py.File(config.get(section, 'matrice_file') if config.has_option(section, 'matrice_file') \
                         else config.get('saving', 'output_file'),'r')
output_name= config.get(section, 'output_name')

experiment_folder = config.get(section, 'experiment_folder') if config.has_option(section, 'experiment_folder') \
                    else config.get('general', 'experiment_folder')
model_name = os.path.join(experiment_folder,  "matrices/" , 'model.099.hdf5')

load_level=config.getint(section, 'load_level') if config.has_option(section, 'load_level') else config.getint('general', 'load_level')
patch_size = config.getint(section, 'patch_size') if config.has_option(section, 'patch_size') else config.getint('general', 'patch_size')
stride = eval(config.get(section, 'stride'))

color_channels = config.getint(section, 'color_channels') if config.has_option(section, 'color_channels') \
                else config.getint('general', 'color_channels')

patch_shape = (patch_size, patch_size,color_channels) # Shape of the image (patch)
patch_mask_shape = (patch_size, patch_size)   # Shape of the mask (patch)

## Image loading

### Loading base picture

In [ ]:
im = op.OpenSlide(filename)
imload = np.asfarray(im.read_region((0,0), load_level, im.level_dimensions[load_level]),dtype=np.float32)[:,:,0:color_channels]/255

In [ ]:
print(im.level_dimensions)

In [ ]:
print(imload.mean())
print(imload.std())

In [ ]:
#normalization
stats = matrice_file['stats'][:]
im_mean=stats[0]
im_std=stats[1]
matrice_file.close()

print(im_mean)
print(im_std)
print(imload.mean())
print(imload.std())

#imload-=imload.mean()
#imload/=imload.std()
imload-=im_mean
imload/=im_std

In [ ]:
print(imload.mean())
print(imload.std())

### Loading mask

In [ ]:
maskload = Image.open(maskname)
maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[load_level]),dtype=np.float32),-1)

### Allocating final array

In [ ]:
outputProba = np.zeros((im.level_dimensions[load_level][1],im.level_dimensions[load_level][0]),dtype=np.float32)
outputProbaTimes = np.zeros((im.level_dimensions[load_level][1],im.level_dimensions[load_level][0]),dtype=np.uint8)

## Loading model

In [ ]:
# Load model
UNet = load_model(model_name)

## Execution

### Prediction

In [ ]:
t0 = time.clock()


# Prediction
for posX in range(0,im.level_dimensions[load_level][0]-patch_size,stride):
    for posY in range(0,im.level_dimensions[load_level][1]-patch_size,stride):

        xx = np.expand_dims(imload[posY:posY+patch_size,posX:posX+patch_size,:],0)
        yy = maskload[posY:posY+patch_size,posX:posX+patch_size,:]
        #xx = xx[:,:,:,0:1]
        pred = UNet.predict(xx)[..., 0].reshape(patch_shape[:2])

        part=outputProba[posY:posY+patch_size,posX:posX+patch_size]
        part[:,:]+=pred[0:part.shape[0],0:part.shape[1]]
        outputProbaTimes[posY:posY+patch_size,posX:posX+patch_size]+=1

t = time.clock()
print("Execution duration : ", t-t0)

### code that follows in progress 

In [ ]:
outputProba.shape

In [ ]:
outputProba/=outputProbaTimes

In [ ]:
np.putmask(outputProba, outputProba != outputProba, 0)


In [ ]:
print(outputProba.min())
print(outputProba.max())

In [ ]:
outputProbaThresh = outputProba > 0.8
maskTresh = maskload > 0.5

In [ ]:
outputProbaThresh = remove_small_regions(outputProbaThresh, 0.02 * np.prod(patch_size))

In [ ]:
testDelAmore = np.dstack((outputProbaThresh.astype(np.int8), maskTresh.astype(np.int8), outputProbaThresh.astype(np.int8)))

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(outputProba,interpolation='none', cmap="jet")
plt.show()

#plt.close()

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(testDelAmore,interpolation='none', cmap="jet")
plt.show()

In [ ]:
mkdirs(experiment_folder + "prediction/")

In [ ]:
misc.imsave(experiment_folder + "prediction/" + output_name, testDelAmore)

In [ ]:
#Trying to save the file on the fly
#maskImage = Image.fromarray(testDelAmore, 'RGB')
#maskImage.save("../data/prediction/prediction_14-02_256_0.9_10-11-13-200-100.png")